In [1]:
library(lavaan)
library(bruceR)
library(tidySEM)

This is lavaan 0.6-19
lavaan is FREE software! Please report any bugs.


bruceR (v2024.6)
Broadly Useful Convenient and Efficient R functions

Packages also loaded:
<U+2714> data.table	<U+2714> emmeans
<U+2714> dplyr     	<U+2714> lmerTest
<U+2714> tidyr     	<U+2714> effectsize
<U+2714> stringr   	<U+2714> performance
<U+2714> ggplot2   	<U+2714> interactions

Main functions of `bruceR`:
cc()          	Describe() 	TTEST()
add()         	Freq()     	MANOVA()
.mean()       	Corr()     	EMMEANS()
set.wd()      	Alpha()    	PROCESS()
import()      	EFA()      	model_summary()
print_table() 	CFA()      	lavaan_summary()

For full functionality, please install all dependencies:
install.packages("bruceR", dep=TRUE)

Online documentation:
https://psychbruce.github.io/bruceR

To use this package in publications, please cite:
Bao, H.-W.-S. (2024). bruceR: Broadly useful convenient and efficient R functions (Version 2024.6) [Computer software]. https://CRAN.R-project.org/package=bruceR



These 

In [4]:
data <- import("data_298.csv", as = "tibble")

In [5]:
baseline <- "
x =~ x1+x2+x3+x4+x5
ma =~ ma1+ma2+ma3+ma4+ma5
mb =~ mb1+mb2+mb3+mb4+mb5+mb6
y =~ y1+y2+y3+y4+y5+y6
"
baseline_model <- cfa(model = baseline, data = data)


three <- "
x =~ x1+x2+x3+x4+x5+
ma1+ma2+ma3+ma4+ma5

mb =~ mb1+mb2+mb3+mb4+mb5+mb6

y =~ y1+y2+y3+y4+y5+y6
"
three_model <- cfa(model = three, data = data)


two <- "
x =~ x1+x2+x3+x4+x5+
ma1+ma2+ma3+ma4+ma5+
mb1+mb2+mb3+mb4+mb5+mb6

y =~ y1+y2+y3+y4+y5+y6
"
two_model <- cfa(model = two, data = data)


one <- "
x =~ x1+x2+x3+x4+x5+
ma1+ma2+ma3+ma4+ma5+
mb1+mb2+mb3+mb4+mb5+mb6+
y1+y2+y3+y4+y5+y6
"
one_model <- cfa(model = one, data = data)


bifactor <- "
x =~ x1+x2+x3+x4+x5
ma =~ ma1+ma2+ma3+ma4+ma5
mb =~ mb1+mb2+mb3+mb4+mb5+mb6
y =~ y1+y2+y3+y4+y5+y6

G =~ x1+x2+x3+x4+x5+
ma1+ma2+ma3+ma4+ma5+
mb1+mb2+mb3+mb4+mb5+mb6+
y1+y2+y3+y4+y5+y6
"
bifactor_model <- cfa(model = bifactor, data = data, orthogonal = TRUE)

In [6]:
# 定义模型列表
models <- list(
  "Baseline model" = baseline_model,
  "Three-factor model" = three_model,
  "Two-factor model" = two_model,
  "Single-factor model" = one_model,
  "Bifactor model" = bifactor_model
)

# 使用 lapply 循环遍历模型，计算拟合指标并整理结果
results <- lapply(names(models), function(model_name) {
  model <- models[[model_name]]

  # 计算拟合指标
  fitted <- table_fit(model)

  # 创建新的数据框并添加模型名称
  fitted |>
    mutate(
      "χ2/df" = chisq / df,
      "χ2" = chisq,
      CFI = cfi,
      TLI = tli,
      RMSEA = rmsea,
      SRMR = srmr,
      Model = model_name
    ) |>
    select(Model, "χ2", df, "χ2/df", CFI, TLI, RMSEA, SRMR)
})

# 将结果合并在一起并输出
final_results <- bind_rows(results)
final_results |> print_table()

─────────────────────────────────────────────────────────────────────
                 Model       χ2      df χ2/df   CFI   TLI RMSEA  SRMR
─────────────────────────────────────────────────────────────────────
1  Baseline model       379.806 203.000 1.871 0.960 0.954 0.054 0.033
2  Three-factor model   577.646 206.000 2.804 0.915 0.905 0.078 0.045
3  Two-factor model     745.921 208.000 3.586 0.877 0.864 0.093 0.055
4  Single-factor model 1019.399 209.000 4.878 0.815 0.796 0.114 0.068
5  Bifactor model       360.755 187.000 1.929 0.960 0.951 0.056 0.032
─────────────────────────────────────────────────────────────────────


In [7]:
# 获取基准模型的模型值
baseline_values <- final_results |>
  filter(Model == "Baseline model")

# 计算其他模型的差值
other_values <- final_results |>
  filter(Model != "Baseline model") |>
  mutate(
    Δχ2 = abs(χ2 - baseline_values$χ2),
    Δdf = abs(df - baseline_values$df),
    p = p.chi2(Δχ2, Δdf)
  )

# 合并并输出结果
model_results <- bind_rows(baseline_values, other_values)
model_results |> print_table()

──────────────────────────────────────────────────────────────────────────────────────────
                 Model       χ2      df χ2/df   CFI   TLI RMSEA  SRMR     Δχ2    Δdf     p
──────────────────────────────────────────────────────────────────────────────────────────
1  Baseline model       379.806 203.000 1.871 0.960 0.954 0.054 0.033                     
2  Three-factor model   577.646 206.000 2.804 0.915 0.905 0.078 0.045 197.840  3.000 0.000
3  Two-factor model     745.921 208.000 3.586 0.877 0.864 0.093 0.055 366.115  5.000 0.000
4  Single-factor model 1019.399 209.000 4.878 0.815 0.796 0.114 0.068 639.593  6.000 0.000
5  Bifactor model       360.755 187.000 1.929 0.960 0.951 0.056 0.032  19.051 16.000 0.266
──────────────────────────────────────────────────────────────────────────────────────────


In [9]:
model_results |> filter(Model == "Baseline model" | Model == "Bifactor model") |> print_table()

───────────────────────────────────────────────────────────────────────────────────
            Model      χ2      df χ2/df   CFI   TLI RMSEA  SRMR    Δχ2    Δdf     p
───────────────────────────────────────────────────────────────────────────────────
1  Baseline model 379.806 203.000 1.871 0.960 0.954 0.054 0.033                    
2  Bifactor model 360.755 187.000 1.929 0.960 0.951 0.056 0.032 19.051 16.000 0.266
───────────────────────────────────────────────────────────────────────────────────
